In [ ]:
# Code needed to produce cutouts used in the pipeline figure

In [ ]:
import numpy as np
from skimage import measure
import os
import pandas as pd
from at_synapse_detection import dataAccess as da
from at_synapse_detection import SynapseDetection as syn
from skimage import exposure
from skimage import color
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm
from importlib import reload
import fxsdata
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
reload(fxsdata)

In [ ]:
fn_psd = '/Users/anish/Documents/yi_mice/2ss_stacks/F000/2ss_PSD.tif'
fn_synapsin = '/Users/anish/Documents/yi_mice/2ss_stacks/F000/2ss_Synap.tif'

In [ ]:
# Create query
query = {'preIF': [fn_synapsin], 'preIF_z': [1],
         'postIF': [fn_psd], 'postIF_z': [1],
         'punctumSize': 2}

In [ ]:
red_hsv = [0, 100/100, 100/100]
green_hsv = [120/360, 100/100, 100/100]
magenta_hsv = [310/360, 100/100, 97/100]
blue_hsv = [240/360, 1, 1]
yellow_hsv = [60/360, 1, 1]
cyan_hsv = [180/360, 1, 1]
white_hsv = [0, 0, 1]

In [ ]:
z = 14

In [ ]:
# Create PSD/Synapsin Figure

In [ ]:
psd_rgb = fxsdata.setup_slice(fn_psd, z)
psd_img = fxsdata.colorize(psd_rgb,green_hsv[0], green_hsv[1], green_hsv[2])

In [ ]:
plt.figure()
plt.imshow(psd_img[750:850, 870:990], interpolation='bilinear')
plt.axis('off')
ax = plt.gca()
fontprops = fm.FontProperties(size=18)

scalebar = AnchoredSizeBar(ax.transData,
                           20, '2 $\mu m$', 'lower right', 
                           pad=0.5,
                           color='white',
                           frameon=False,
                           size_vertical=1, 
                          fontproperties=fontprops)

ax.add_artist(scalebar)
plt.savefig('figs/raw-data/psd_2ss_f000.png', dpi=300, bbox_inches='tight')

In [ ]:
synapsin_rgb = fxsdata.setup_slice(fn_synapsin, z)
synapsin_img = fxsdata.colorize(synapsin_rgb, magenta_hsv[0], magenta_hsv[1], magenta_hsv[2])

In [ ]:
plt.figure()
plt.imshow(synapsin_img[750:850, 870:990], interpolation='bilinear')
plt.axis('off')
ax = plt.gca()
fontprops = fm.FontProperties(size=18)

scalebar = AnchoredSizeBar(ax.transData,
                           20, '2 $\mu m$', 'lower right', 
                           pad=0.5,
                           color='white',
                           frameon=False,
                           size_vertical=1, 
                          fontproperties=fontprops)

ax.add_artist(scalebar)
plt.savefig('figs/raw-data/synapsin_2ss_f000.png', dpi=300, bbox_inches='tight')

In [ ]:
synaptic_volumes = da.load_tiff_from_query(query)
resultvol = syn.getSynapseDetections(synaptic_volumes, query)

In [ ]:
resultimg = resultvol[750:850, 870:990, z]
bwimg = resultimg > 0.9

In [ ]:
resultmap_img = color.gray2rgb(resultimg)
resultmap_img = fxsdata.colorize(resultmap_img, white_hsv[0], white_hsv[1], white_hsv[2])

In [ ]:
plt.figure()
plt.imshow(resultmap_img, interpolation='bilinear')
plt.axis('off')
ax = plt.gca()
fontprops = fm.FontProperties(size=18)

scalebar = AnchoredSizeBar(ax.transData,
                           20, '2 $\mu m$', 'lower right', 
                           pad=0.5,
                           color='red',
                           frameon=False,
                           size_vertical=1, 
                          fontproperties=fontprops)

ax.add_artist(scalebar)
plt.savefig('figs/raw-data/result_2ss_f000.png', dpi=300, bbox_inches='tight')

In [ ]:
detections_img = color.gray2rgb(bwimg)
detections_img = fxsdata.colorize(detections_img, white_hsv[0], white_hsv[1], white_hsv[2])

In [ ]:
combined_channels=np.nanmax([psd_img[750:850, 870:990], synapsin_img[750:850, 870:990], detections_img], axis=0)
maxints = tuple(np.nanmax(combined_channels[:,:,i]) for i in [0,1,2])

for i in [0,1,2]: 
    combined_channels[:,:,i]=exposure.rescale_intensity(combined_channels[:,:,i], 
                        out_range=(0, combined_channels[:,:,i].max()/np.nanmax(maxints)));

In [ ]:
plt.figure()
plt.imshow(combined_channels, interpolation='bilinear')
plt.axis('off')
ax = plt.gca()
fontprops = fm.FontProperties(size=18)

scalebar = AnchoredSizeBar(ax.transData,
                           20, '2 $\mu m$', 'lower right', 
                           pad=0.5,
                           color='red',
                           frameon=False,
                           size_vertical=1, 
                          fontproperties=fontprops)

ax.add_artist(scalebar)
plt.savefig('figs/raw-data/results_overlaid_2ss_f000.png', dpi=300, bbox_inches='tight')